In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
from combine import recombine
data = recombine("../data/potholes_1.csv", "../data/potholes_2.csv")
data["creation_date"] = pd.to_datetime(data["CREATION DATE"])
data["completion_date"] = pd.to_datetime(data["COMPLETION DATE"])
data["creation_year"] = pd.DatetimeIndex(data["creation_date"]).year

# Convert strings to times
data["creation_date"] = pd.to_datetime(data["creation_date"], format = "%m/%d/%Y")
data["completion_date"] = pd.to_datetime(data["completion_date"], format = "%m/%d/%Y")

# Create completion time variable
data["completion_time"] = data["completion_date"] - data["creation_date"]

data.columns

Index(['CREATION DATE', 'STATUS', 'COMPLETION DATE', 'SERVICE REQUEST NUMBER',
       'TYPE OF SERVICE REQUEST', 'CURRENT ACTIVITY', 'MOST RECENT ACTION',
       'NUMBER OF POTHOLES FILLED ON BLOCK', 'STREET ADDRESS', 'ZIP',
       'X COORDINATE', 'Y COORDINATE', 'Ward', 'Police District',
       'Community Area', 'SSA', 'LATITUDE', 'LONGITUDE', 'LOCATION',
       'creation_date', 'completion_date', 'creation_year', 'completion_time'],
      dtype='object')

In [3]:
data["completion_time"] = data["completion_time"].apply(lambda x: x.days)

In [4]:
data.drop('CREATION DATE', axis=1, inplace=True)
data.drop('COMPLETION DATE', axis=1, inplace=True)
data.drop('STATUS', axis=1, inplace=True)
data.drop('SERVICE REQUEST NUMBER', axis=1, inplace=True)
data.drop('TYPE OF SERVICE REQUEST', axis=1, inplace=True)
data.drop('CURRENT ACTIVITY', axis=1, inplace=True)
data.drop('MOST RECENT ACTION', axis=1, inplace=True)
data.drop('STREET ADDRESS', axis=1, inplace=True)
data.drop('X COORDINATE', axis=1, inplace=True)
data.drop('Y COORDINATE', axis=1, inplace=True)
data.drop('SSA', axis=1, inplace=True)
data.drop('LATITUDE', axis=1, inplace=True)
data.drop('LONGITUDE', axis=1, inplace=True)
data.drop('LOCATION', axis=1, inplace=True)
data.drop('creation_date', axis=1, inplace=True)
data.drop('completion_date', axis=1, inplace=True)
data.drop('ZIP', axis=1, inplace=True)
data.drop('Ward', axis=1, inplace=True)
data.drop('Police District', axis=1, inplace=True)
data.drop('NUMBER OF POTHOLES FILLED ON BLOCK', axis=1, inplace=True)

data["community_area"] = pd.to_datetime(data["Community Area"])
data.drop('Community Area', axis=1, inplace=True)

data.columns

Index(['creation_year', 'completion_time', 'community_area'], dtype='object')